In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from sklearn.model_selection import train_test_split
import numpy as np
import pywt
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.linear_model import LogisticRegression
from scipy import stats 
import os
from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing
from keras.utils import np_utils

In [ ]:
scaler= StandardScaler()
def data_preprocess_train(X):
    X_prep=scaler.fit_transform(X)
    #do here your preprocessing
    return X_prep

def feature_normalize(dataset):
    mu = np.mean(dataset, axis=0)
    sigma = np.std(dataset, axis=0)
    return (dataset - mu)/sigma
 
def show_confusion_matrix(validations, predictions):

    matrix = metrics.confusion_matrix(validations, predictions)
    plt.figure(figsize=(6, 4))
    sns.heatmap(matrix,
                cmap="coolwarm",
                linecolor='white',
                linewidths=1,
                xticklabels=LABELS,
                yticklabels=LABELS,
                annot=True,
                fmt="d")
    plt.title("Confusion Matrix")
    plt.ylabel("True Label")
    plt.xlabel("Predicted Label")
    plt.show()
    return

def show_basic_dataframe_info(dataframe,
                              preview_rows=20):

    """
    This function shows basic information for the given dataframe

    Args:
        dataframe: A Pandas DataFrame expected to contain data
        preview_rows: An integer value of how many rows to preview

    Returns:
        Nothing
    """

    # Shape and how many rows and columns
    print("Number of columns in the dataframe: %i" % (dataframe.shape[1]))
    print("Number of rows in the dataframe: %i\n" % (dataframe.shape[0]))
    print("First 20 rows of the dataframe:\n")
    # Show first 20 rows
    print(dataframe.head(preview_rows))
    print("\nDescription of dataframe:\n")
    # Describe dataset like mean, min, max, etc.
    # print(dataframe.describe())


def read_data(file_path):

    """
    This function reads the EEG data from a file

    Args:
        file_path: URL pointing to the CSV file

    Returns:
        A pandas dataframe
    """

    column_names = ['code',
                    'id-user',
                    'time-stamp',
                    'sample',
                    'canal1',
                    'canal2',
                    'canal3',
                    'canal4',
                    'canal5',
                    'canal6',
                    'canal7',
                    'canal8',
                    'canal9',
                    'canal10',
                    'canal11',
                    'canal12',
                    'canal13',
                    'canal14',
                    'canal15',
                    'canal16',
                    'canal17',
                    'canal18',
                    'canal19',
                    'canal20',
                    'canal21',
                    'canal22',  
                    'canal23',
                    'canal24',
                    'canal25',
                    'canal26',
                    'canal27',
                    'canal28',
                    'canal29',
                    'canal30',
                    'canal31',
                    'canal32',
                    'canal33',
                    'canal34',
                    'canal35',
                    'canal36',
                    'canal37',
                    'canal38',
                    'canal39',
                    'canal40',
                    'canal41',
                    'canal42'              
                    ]
    df = pd.read_csv(file_path,
                     header=None,
                     names=column_names)
    return df

def shuffle_in_unison_scary(a, b):
    rng_state = np.random.get_state()
    np.random.shuffle(a)
    np.random.set_state(rng_state)
    np.random.shuffle(b)
def create_segments_and_labels(df, time_steps, step, label_name):

    """
    This function receives a dataframe and returns the reshaped segments
    of eeg canals as well as the corresponding labels

    Args:
        df: Dataframe in the expected format
        time_steps: Integer value of the length of a segment that is created
    Returns:
        reshaped_segments
        labels:
    """

   
    global N_CANALS
    # Number of steps to advance in each iteration (for me, it should always
    # be equal to the time_steps in order to have no overlap between segments)
    # step = time_steps
    
    segments = []
    labels = []
    i=0
    #for i in range(0, len(df) - time_steps, step):
    while i<len(df) - time_steps:
        oneSegment = []    
        for j in range(1,N_CANALS+1):
             
            xx = df['canal'+str(j)].values[i: i + time_steps]
            oneSegment.append(xx)
           
        arr=np.asarray(oneSegment,dtype= np.float32)
        arr=data_preprocess_train(arr)
        arr=feature_normalize(arr)
        segments.append(arr)
        label = stats.mode(df[label_name][i: i + time_steps])[0][0]
        labels.append(label)
        i=i+step
     
    shuffle_in_unison_scary(segments,labels)
    reshaped_segments = np.asarray(segments).reshape(-1, time_steps, N_CANALS)
    labels = np.asarray(labels)
    return reshaped_segments, labels

In [ ]:
LABELS = ["1",
          "2",
          "3",
          "4",
          "5"]


N_CANALS = 42
# The number of steps within one time segment
TIME_PERIODS = 750 #3 seunds per 250 samples after stimulation

STEP_DISTANCE = 750

In [ ]:
df = read_data('/kaggle/input/eeg-from-motor-cortex/dataKeras.csv')

# Describe the data
show_basic_dataframe_info(df, N_CANALS)

# Define column name of the label vector


# %%

print("\n--- Reshape the data into segments ---\n")

# Differentiate between test set and training set
del df['time-stamp']
del df['sample']

In [ ]:
LABEL = "code"
le = preprocessing.LabelEncoder()
# Add a new column to the existing DataFrame with the encoded values
df[LABEL] = le.fit_transform(df["code"].values.ravel())
df_test = df[df['id-user'] > 7]
df_train = df[df['id-user'] <= 7]

x_train, y_train = create_segments_and_labels(df_train,
                                              TIME_PERIODS,
                                              STEP_DISTANCE,
                                              LABEL)
print('x_train shape: ', x_train.shape)

print(x_train.shape[0], 'training samples')


# Inspect y data
print('y_train shape: ', y_train.shape)

num_time_periods, num_sensors = x_train.shape[1], x_train.shape[2]
print("num_sensors "+str(num_sensors))
num_classes = le.classes_.size
print("The OpenVibe stimulationes code, from http://openvibe.inria.fr/stimulation-codes/")
print(list(le.classes_))

y_train = np_utils.to_categorical(y_train, num_classes)
print('New y_train shape: ', y_train.shape)

In [ ]:
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Reshape, GlobalAveragePooling1D,GlobalMaxPooling1D
from keras.layers import Conv1D, MaxPooling1D,AveragePooling1D
from keras.layers import  BatchNormalization,  Flatten, ReLU, LSTM 
from keras.utils import np_utils
from keras.optimizers import Adam
from keras.callbacks import LearningRateScheduler

seed = 3421
tf.random.set_seed(seed)
np.random.seed(seed)

model_m = Sequential()
#model_m.add(Reshape((TIME_PERIODS, num_sensors), input_shape=(input_shape,)))
model_m.add(Conv1D(64,7, activation='relu',strides = 3, input_shape=(TIME_PERIODS, num_sensors)))
model_m.add(Conv1D(64,7, activation='relu'))
#model_m.add(MaxPooling1D(3))

model_m.add(BatchNormalization())
model_m.add(Dropout(0.2))
model_m.add(Conv1D(128, 5,strides = 2, activation='relu'))
model_m.add(Conv1D(128, 5, activation='relu'))
#model_m.add(MaxPooling1D(3))
model_m.add(BatchNormalization())
model_m.add(Dropout(0.2))
model_m.add(Conv1D(256, 3, activation='relu'))
model_m.add(Conv1D(256, 3, activation='relu'))
#model_m.add(GlobalMaxPooling1D())
#model_m.add(GlobalAveragePooling1D())
#model_m.add(MaxPooling1D(3))

model_m.add(BatchNormalization())
model_m.add(Dropout(0.2))
model_m.add(Flatten())
model_m.add(BatchNormalization())
#model_m.add(Dropout(0.2))
#model_m.add(Dense(42, activation = "softmax"))
#model_m.add(BatchNormalization())
#model_m.add(Dropout(0.2))
model_m.add(Dense(num_classes, activation='softmax'))
#model_m.add(Dense(num_classes, activation = "sigmoid"))
model_m.summary()

In [ ]:
callbacks_list = [
    keras.callbacks.ModelCheckpoint(
        filepath='best_model.{epoch:02d}-{loss:.2f}.h5',
        monitor='loss', save_best_only=True),
    keras.callbacks.EarlyStopping(monitor='acc', patience=10)
]
adam = Adam(learning_rate = 0.0001)
model_m.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['acc'])


In [ ]:
BATCH_SIZE = 100
EPOCHS =50

history = model_m.fit(x_train,
                      y_train,
                      batch_size=BATCH_SIZE,
                      epochs=EPOCHS,
                      callbacks=callbacks_list,
                      validation_split=0.2,
                      verbose=1)

In [ ]:
print("\n--- Learning curve of model training ---\n")

# summarize history for accuracy and loss

plt.figure(figsize=(6, 4))
plt.plot(history.history['acc'], "g--", label="Accuracy of training data")
plt.plot(history.history['val_acc'], "g", label="Accuracy of validation data")
plt.plot(history.history['loss'], "r--", label="Loss of training data")
plt.plot(history.history['val_loss'], "r", label="Loss of validation data")
plt.title('Model Accuracy and Loss')
plt.ylabel('Accuracy and Loss')
plt.xlabel('Training Epoch')
plt.ylim(0)
plt.legend()
plt.show()

In [ ]:
import seaborn as sns
from sklearn import metrics
from sklearn.metrics import classification_report

for i in range(1,N_CANALS+1):
    df_test['canal'+str(i)] = feature_normalize(df_test['canal'+str(i)])

x_test, y_test = create_segments_and_labels(df_test,
                                            TIME_PERIODS,
                                            STEP_DISTANCE,
                                            LABEL)

x_test = x_test.astype("float32")
y_test = y_test.astype("float32")

y_test = np_utils.to_categorical(y_test, num_classes)

score = model_m.evaluate(x_test, y_test, verbose=1)

print("\nAccuracy on test data: %0.2f" % score[1])
print("\nLoss on test data: %0.2f" % score[0])

# %%

print("\n--- Confusion matrix for test data ---\n")

y_pred_test = model_m.predict(x_test)
# Take the class with the highest probability from the test predictions
max_y_pred_test = np.argmax(y_pred_test, axis=1)
max_y_test = np.argmax(y_test, axis=1)

show_confusion_matrix(max_y_test, max_y_pred_test)